In [ ]:
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.layers as tkl
import tensorflow as tf
from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D, Dense, Input, Conv2D, UpSampling2D, BatchNormalization, Activation, Add
from tensorflow.keras.optimizers import Adam
from keras.callbacks import EarlyStopping
import cv2
from sklearn.model_selection import train_test_split
from skimage.util import random_noise
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import glob
import random
from itertools import chain

In [ ]:
WIDTH = 256
HEIGHT = 256
n_channels = 3
BATCH_SIZE = 32 

In [ ]:
def identical_block(X, n_filters, kernel_size):
  x_shortcut = X
  
  '''first part of identify block: '''

  res_block = Conv2D(filters = n_filters, 
               kernel_size = kernel_size, padding = 'same')(X)
  res_block = BatchNormalization()(res_block)
  res_block = Activation('relu')(res_block)
  
  '''Second part of identify block: '''

  res_block = Conv2D(filters = n_filters, 
               kernel_size = kernel_size, padding = 'same')(res_block)
  res_block = BatchNormalization()(res_block)
  
  ''' skip part: '''

  res_block = Add()([res_block, x_shortcut])
  # res_block = Activation('relu')

  return res_block


# def conv_block(inputs, n_filters = 64, kernel_size = (3,3)):
  
#   conv = tkl.Conv2D(n_filters, kernel_size)(inputs)
#   conv = tkl.BatchNormalization()(conv)
#   conv = tkl.Activation('relu')(conv)
#   conv = tkl.Conv2D(n_filters, kernel_size)(conv)
#   conv = tkl.BatchNormalization()(conv)

#   return conv


def encoder(input, skip, n_filters, kernel_size):
  input = input
  
  pix_shuffle = tf.nn.depth_to_space(input, 4, data_format='NHWC')
  stacked = tf.stack([skip, pix_shuffle])

  enc_block = BatchNormalization()(stacked)
  enc_block = Conv2D(filters=n_filters, kernel_size=kernel_size)(enc_block)
  enc_block = Activation('relu')(enc_block)
  enc_block = Conv2D(filters=n_filters, kernel_size=kernel_size)(enc_block)
  enc_block = Activation('relu')(enc_block)
  # enc_block = Activation('relu')(enc_block)

  return enc_block


def r_unet():

  ''' Contraction path (Encoder): '''

  input_shape = (WIDTH, HEIGHT, n_channels)
  input_img = tf.keras.Input(input_shape, name = 'image')

  first_conv = Conv2D(filters=64, kernel_size=(7,7))(input_img)
  first_conv = BatchNormalization()(first_conv)
  first_conv = Activation('relu')(first_conv)
  skip0 = first_conv
  out0 = MaxPooling2D()(first_conv)

  res_block = identical_block(out0, 64, (3,3))
  res_block = identical_block(res_block, 64, (3,3))
  res_block = identical_block(res_block, 64, (3,3))
  res_block = identical_block(res_block, 128, (3,3))
  skip1 = res_block
  out1 = MaxPooling2D()(res_block)
  
  res_block = identical_block(out1, 128, (3,3))
  res_block = identical_block(res_block, 128, (3,3))
  res_block = identical_block(res_block, 128, (3,3))
  res_block = identical_block(res_block, 256, (3,3))
  skip2 = res_block
  out2 = MaxPooling()(res_block)

  res_block = identical_block(out2, 256, (3,3))
  res_block = identical_block(res_block, 256, (3,3))
  res_block = identical_block(res_block, 256, (3,3))
  res_block = identical_block(res_block, 256, (3,3))
  res_block = identical_block(res_block, 256, (3,3))
  res_block = identical_block(res_block, 512, (3,3))
  skip3 = res_block
  out3 = MaxPooling()(res_block)

  res_block = identical_block(out3, 512, (3,3))
  res_block = identical_block(res_block, 512, (3,3))
  conv = BatchNormalization()(res_block)
  conv = Activation('relu')(conv)
  skip4 = conv
  
  conv = Conv2D(1024, (3,3))(conv)
  conv = Activation('relu')(conv)
  conv = Conv2D(512, (3,3))(conv)
  conv = Activation('relu')(conv)


  ''' Expansion path (Decoder): '''

  enc_block = encoder(conv, skip4, 512, (3,3))
  enc_block = encoder(input, skip3, 256, (3,3))
  enc_block = encoder(enc_block, skip2, 128, (3,3))
  enc_block = encoder(enc_block, skip1, 128, (3,3))
  
  enc_block = tf.nn.depth_to_space(enc_block, 2, data_format='NHWC')
  stacked = tf.stack([skip0, enc_block])

  last_conv = Conv2D(64, (3,3))(stacked)
  last_conv = Activation('relu')(last_conv)
  last_conv = Conv2D(32, (3,3))(stacked)
  last_conv = Activation('relu')(last_conv)
  output = Conv2D(3, (1,1), activation='sigmoid')(last_conv)
  
  model = tf.keras.Model(inputs = [input_img], outputs = [output])

  return model

  

In [ ]:
model = r_unet()

ValueError: ignored